In [173]:
# importing dependencies
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize

In [82]:
#stop word removal
from nltk.corpus import stopwords
stop={'veya', 'çünkü', 'olarak', 'kadar ', 'ise', 'bir', 'de', 'ile', 'için','ile', 'yaklaşık', 'e karşı', 'arasında', 'içine', 'ile', 'sırasında', 'önce', 'sonra', 'yukarıdaki', 'aşağıda', 'için', 'dan', 'yukarı', 'aşağı', 'içinde', 'dışarı', 'açık', 'kapalı', 'üzerinde ' 'başlığı altında', 'tekrar', 'ileri', 'daha sonra', 'bir zamanlar', 'burada', 'orada', 'ne zaman', 'nerede', 'niçin', 'nasıl', 'bütün', 'herhangi', 'hem', 'her', 'birkaç', 'daha', 'en', 'öteki', 'bazı', 'yalnızca', 'kendi', 'aynı', 'bu yüzden', 'daha ', 'çok', 'çok', 'irade', 'sadece', 'gerektiğini', 'şimdi' } #add related stop word here
print(stop)

{'ne zaman', 'bazı', 'irade', 'yukarıdaki', 'kendi', 'de', 'daha ', 'nasıl', 'dan', 'daha', 'çünkü', 'sonra', 'niçin', 'öteki', 'bu yüzden', 'aşağı', 'arasında', 'yaklaşık', 'aşağıda', 'içinde', 'aynı', 'gerektiğini', 'şimdi', 'bir zamanlar', 'e karşı', 'içine', 'ile', 'dışarı', 'nerede', 'olarak', 'önce', 'her', 'ise', 'bir', 'sırasında', 'orada', 'tekrar', 'ileri', 'sadece', 'üzerinde başlığı altında', 'burada', 'birkaç', 'yalnızca', 'kapalı', 'kadar ', 'yukarı', 'çok', 'bütün', 'açık', 'veya', 'herhangi', 'daha sonra', 'hem', 'en', 'için'}


In [174]:
df_csv = pd.read_csv('Turkish/offenseval-tr-training-v1.tsv',sep='\t')
df_test = pd.read_csv('Turkish/offenseval-tr-testset-v1.tsv',sep='\t')

In [175]:
#marge all data from files that has been read
twee=[]
ids=[]
for i in range(len(df_test)):
    ids.append(df_test['id'][i])
    twee.append(df_test['tweet'][i])

#marge all data from files that has been read

q_a=[]
label=[]
for i in range(len(df_csv)):
    label.append(df_csv['subtask_a'][i])
    q_a.append(df_csv['tweet'][i])

In [176]:
print(len(q_a))
print(len(label))
print(len(twee))
print(len(ids))

31277
31277
3528
3528


In [177]:
# convert labeling to 0 and 1
lb=[]
for i in label:
    if i=='OFF':
        lb.append(1)
    if i=='NOT':
        lb.append(0)

print(lb)
label=lb

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 

In [156]:
# remove empty space from the begining of a sentance
import emoji
QA=[]
for sen in q_a:
#     sen=i.replace('@USER','')
#     sen=sen.replace('@user','')
#     sen=sen.replace("#", ' ')
    sen=emoji.demojize(sen)
    sen=" ".join(sen.split())
    QA.append(sen)
print(QA)
q_a=QA

["@USER en güzel uyuyan insan ödülü jeon jungkook'a gidiyor...", '@USER Mekanı cennet olsun, saygılar sayın avukatımız,iyi günler dilerim', 'Kızlar aranızda kas yığını beylere düşenler olduğunu görüyorum yapmayın allah aşkına', 'Biraz ders çalışayım. Tembellik ve uyku düşmanımızdır.', '@USER Trezeguet yerine El Sharawy daha iyi olmaz mı', '@USER Bence de olması gerekiyor. Hatta meslek lisesine ön hazırlık olarak ortaokulda bile çıraklık tarzında eğitimler verilip temeli hazırlanmalıdır. Lise çağına gelen bir çocuk için bir çok konuda vakit geçmiş oluyor. O nedenle de meslek liselerinde istenilen performans yakalanamıyor.', 'Mutlu gorunumlu ama daima mutsuz olanlar burdami? Mutlu olun olmaya calisin sizi mutlu edecek seyler yapin cok dusunurseniz mutsuz olursunuz hayati akisina birakin zaten olacak birsey olacaksa birgun elbet olur siz yinede mutlu olun.', '100 liraya tras oldum arkadaşım diyo ki ne kesiyo o kurban mı kesiyo agshsn', '@USER @USER @USER Reis bu ülkenin Devlet Başkanı ve 

In [60]:
from string import ascii_lowercase
TWEETS=[]
for sen in twee:
#     sen=i.replace('@USER','')
#     sen=sen.replace('@user','')
#     sen=sen.replace("#", ' ')
    sen=emoji.demojize(sen)
    sen=" ".join(sen.split())
    TWEETS.append(sen)
twee=TWEETS

In [73]:
# for i in twee:
#     print(i)

In [142]:
# #this code will stem text
# stem_both_q_a=[]
# for i in q_a:
#     ps = PorterStemmer() #stemming word removing
#     words = str(i).split(' ') 
#     s=''
#     for w in words: 
#         stem=ps.stem(w)
#         s=s+' '+stem
#     s=s.strip()

#     stem_both_q_a.append(s)
# q_a=stem_both_q_a

# print(q_a)

In [178]:
# # Read data files
comments_attack=pd.DataFrame()
comments_attack['comment']=q_a
comments_attack['label']=label

In [196]:
dataframe = comments_attack

In [197]:
# Text preprocessing
from string import ascii_lowercase
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace('@USER',''))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("_", ""))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(":", ""))
# for i in ascii_lowercase:
#     dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(i, ""))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.lower())
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[/(){}\[\]\|@,;]','',x)))
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[^0-9a-z #+_]','',x)))
# dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub(' +',' ',x)))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" w ", ' with '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" mf ", ' motherfucker '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" IDGAF ", ' i dont give a fuck '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" IDRC ", ' I Dont Really Care '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" tf ", ' the fuck '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" wtf ", ' what the fuck '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" yanno ", ' you know '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" igt ", ' i know right '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" r ", ' are '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" tbh ", ' to be honest '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" ik ", ' i know '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" af ", ' as fuck '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" des ", ' this '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" bout ", ' about '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" em ", ' them '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" stg ", ' swear to god '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" bj ", ' blow job '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" ig ", ' i guess '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" fagg ", ' faggot '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" fag ", ' faggot '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" tfw ", ' That Feel When '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" wa ", ' was '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" n ", ' and '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" y0 ", ' you '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" dat ", ' that '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" YO ", ' you '))



In [198]:
# this code will remove stop word
for c in stop: #stop word removing
    dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(' '+c+' ', ' '))


In [188]:
print(dataframe['label'])

0        0
1        0
2        0
3        0
4        0
        ..
31272    0
31273    0
31274    1
31275    0
31276    0
Name: label, Length: 31277, dtype: int64


In [49]:
comments_attack['label'][990]==0

True

In [199]:
print(dataframe['comment'])

0              gzel uyuyan insan dl jeon jungkooka gidiyor
1         mekan cennet olsun sayglar sayn avukatmziyi g...
2        kzlar aranzda kas yn beylere denler olduunu gr...
3              biraz ders alaym tembellik ve uyku dmanmzdr
4                  trezeguet yerine el sharawy iyi olmaz m
                               ...                        
31272    bu dl sunan kz kim giydirdiyse kzn byk dman o ...
31273    bunu sana beddua etmiyorum bunlar ilerde seni ...
31274    chpliler sandklar brakmyor stne oturmu civciv ...
31275              karanln iinde yalnz kalsam ne oluuuuurr
31276     ne yalan syleyeyim bu haftalkta fitil grebile...
Name: comment, Length: 31277, dtype: object


In [201]:
#split the data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(dataframe['comment'], dataframe['label'], test_size=0.01, random_state=42)

In [202]:
print('train comments length: ',len(train_x))
print('test comments length: ',len(valid_x))

train comments length:  30964
test comments length:  313


In [203]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(dataframe['comment'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# # ngram level tf-idf 
# tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(3,4), max_features=5000)
# tfidf_vect_ngram.fit(dataframe['comment'])
# xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
# xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# # characters level tf-idf
# tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
# tfidf_vect_ngram_chars.fit(dataframe['comment'])
# xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
# xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)


In [148]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(dataframe['comment'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure e
# qual length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [204]:
def train_model(classifier, xtrain, ytrain, xvalid, yvalid): 
    # fit the training dataset on the classifier
    classifier.fit(xtrain, ytrain)
    # predict the labels on validation dataset
    predictions = classifier.predict(xvalid)     
    accuracy = metrics.accuracy_score(predictions, yvalid)
    f1score = metrics.f1_score(yvalid, predictions, average='weighted')
    return accuracy, f1score

In [133]:
# # Naive Bayes on Word Level TF IDF Vectors
# accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
# print("NB, WordLevel TF-IDF:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# # Naive Bayes on Ngram Level TF IDF Vectors
# accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
# print("NB, N-Gram Vectors:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# # Naive Bayes on Character Level TF IDF Vectors
# accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
# print("NB, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

In [205]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("LR, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# # Linear Classifier on Ngram Level TF IDF Vectors
# accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
# print("LR, N-Gram Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# # Linear Classifier on Character Level TF IDF Vectors
# accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
# print("LR, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

LR, WordLevel TF-IDF:   accuracy: 0.8402555910543131   f1 score: 0.8083956208100435


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [73]:
# convert to one_hot
train_y_onehot = keras.utils.to_categorical(train_y, 3)
valid_y_onehot = keras.utils.to_categorical(valid_y, 3)

In [74]:
def cnn(xtrain, ytrain, xvalid, yvalid, epochs = 3):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=epochs)
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [75]:
accuracy, f1score = cnn(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("CNN, Word Embeddings acuuracy accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

NameError: name 'train_seq_x' is not defined

In [130]:
def lstm(xtrain, ytrain, xvalid, yvalid, epochs = 1):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer1 = layers.LSTM(128)(embedding_layer)
    dropout1 = layers.Dropout(0.5)(lstm_layer1)
    #lstm_layer2 = layers.LSTM(128)(dropout1)
    #dropout2 = layers.Dropout(0.5)(lstm_layer2)
    # Add the output Layers
    output_layer = layers.Dense(3, activation="softmax")(dropout1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=3)
    
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [131]:
accuracy, f1score = lstm(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("LSTM, Word Embeddings accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

Epoch 1/3
23887/23887 [==============================] - 42s 2ms/step - loss: 0.4001 - accuracy: 0.8336
Epoch 2/3
23887/23887 [==============================] - 45s 2ms/step - loss: 0.2649 - accuracy: 0.8861
Epoch 3/3
23887/23887 [==============================] - 48s 2ms/step - loss: 0.2501 - accuracy: 0.8898
LSTM, Word Embeddings accuracy:0.8933355808258057     f1 score: 0.8889655234543239


In [159]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [163]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print(accuracy)

# Naive Bayes on Word Level TF IDF Vectors

accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print(accuracy)

# RF on Word Level TF IDF Vectors ngram
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print(accuracy)

# RF on Word Levelon Character Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)

print(accuracy)






C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8003516624040921


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8005115089514067


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7937979539641944


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8254475703324808


In [76]:
clf = Pipeline([
    ('vect',tfidf_vect_ngram_chars),
    ('clf', linear_model.LogisticRegression()),
])

clf = clf.fit(train_x, train_y)

C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [79]:
Labels=[]
IDE=[]
for i in range(len(twee)):
    IDE.append(ids[i])
    predictions = clf.predict([twee[i]])
    if predictions[0]==1:
        Labels.append('OFF')
    elif predictions[0]==0:
        Labels.append('NOT')
print(len(Labels))
print(len(IDE))

3528
3528


In [81]:
my_dict = { 'q_a' : IDE,'label' : Labels}
df = pd.DataFrame(my_dict)
df.to_csv('turkish_',index=False)

In [15]:
df_csv = pd.read_csv('submission_turkish.csv',sep=',')
df_test = pd.read_csv('Turkish/task_a_baseline.csv',sep=',')

In [23]:
for i in range(len(df_csv)):
    x=df_csv['41993'][i]
    y=df_test['41993'][i]
    if x!=y:
        print(y)
    

18715
14763
39586
21605
24763
26560
21998
12859
28251
17393
22663
35895
21969
18385
24498
34620
28333
33928
16101
24772
29714
22548
44115
14316
45593
17339
12213
27490
37172
35498
36755
27805
23848
23178
16803
40360
13794
36415
44559
13700
14501
29386
23572
31510
20720
31350
43404
34687
22784
36254
10835
31638
31067
28311
42065
18670
38429
13486
28191
18712
43797
12587
15235
32949
31043
21397
15764
26812
31700
26639
40024
40597
11316
18067
32170
22161
26124
22723
41139
26775
32139
34184
45561
32874
16459
39154
21719
26808
27869
29481
45002
41888
23865
14544
29370
22552
17039
35675
30098
37178
11465
25170
43870
10138
27120
34091
14698
13755
41811
44395
30519
13916
44296
14922
34594
26256
26580
39064
14150
22827
25763
27394
18473
27145
34314
40914
12096
17157
16670
10655
34194
45505
13148
33718
20435
41301
43066
42169
40936
41374
19402
20374
13030
12006
13945
33504
24030
12712
20850
19880
38203
38014
25521
41264
20576
28310
28441
13801
37266
27963
19305
19358
38116
35570
32643
41785
1604

34817
32969
23531
41159
36278
34423
43471
31959
45627
30510
43185
28096
35919
36710
32581
30315
35560
43563
35377
13962
43914
41365
39366
35739
28346
34365
25157
33114
30184
32608
22490
10047
23527
14206
28266
42045
38212
23568
30592
11190
13593
23789
26231
31309
12738
40382
37265
13599
35009
32153
40676
26447
13748
26597
33919
29715
37813
32220
45808
37762
28489
34916
21011
10295
18938
31817
44059
22171
26211
13155
29131
44259
11661
29690
35263
17359
22944
17100
23716
27310
44845
29572
31720
45005
41773
18798
17461
10082
39464
14376
10664
22321
11839
11899
31645
36988
31860
23106
41103
24581
31052
30618
46026
28701
13740
14903
10125
25198
20657
45154
17995
31062
16511
24883
20133
45742
28576
25725
16275
20303
40521
19603
24253
12718
13010
17797
43443
35335
16760
38068
26390
41370
32651
10191
40628
42350
34929
39863
20453
32890
33531
40734
29254
42518
37658
19135
12029
19846
38841
38860
18206
40702
18611
11476
13701
42876
42075
40662
16798
17804
10527
18593
19853
41961
33635
32731
4021